In [1]:
import tensorflow as tf
import pandas as pd
tf.__version__

'2.13.0'

In [2]:
# Input data
df = pd.read_csv('../input_data/CSVs/Bias_correction_ucl.csv')
# Drop dates and columns with null values
df.drop('Date', inplace=True, axis=1)
df.dropna(inplace=True)
df

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,...,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,...,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,...,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,...,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,...,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,21.0,23.1,17.8,24.688997,78.261383,27.812697,18.303014,6.603253,9.614074,0.052902,...,0.0,0.0,0.0,37.5507,127.040,26.2980,0.5721,4456.024414,27.6,17.7
7746,22.0,22.5,17.4,30.094858,83.690018,26.704905,17.814038,5.768083,82.146707,0.066461,...,0.0,0.0,0.0,37.5102,127.086,21.9668,0.1332,4441.803711,28.0,17.1
7747,23.0,23.3,17.1,26.741310,78.869858,26.352081,18.775678,6.148918,72.058294,0.030034,...,0.0,0.0,0.0,37.5372,126.891,15.5876,0.1554,4443.313965,28.3,18.1
7748,24.0,23.3,17.7,24.040634,77.294975,27.010193,18.733519,6.542819,47.241457,0.035874,...,0.0,0.0,0.0,37.5237,126.909,17.2956,0.2223,4438.373535,28.6,18.8


In [3]:
# Create target set as future temps, and feature set as all data with suture temps removed
target = df[['Next_Tmax', 'Next_Tmin']]
features = df.drop(['Next_Tmax', 'Next_Tmin'], axis=1)

In [4]:
from sklearn.preprocessing import MinMaxScaler
# Rescale the feature set to be between 0 and 1
scaler = MinMaxScaler()
feature_array = scaler.fit_transform(features)
features = pd.DataFrame(feature_array, columns=features.columns)

In [5]:
# Initialising the model
model = tf.keras.Sequential()
# Add an input layer with shape matching the feature set
model.add(tf.keras.layers.InputLayer(input_shape=(features.shape[1],), name='Input_layer'))

# Add 3 dense layers for inner computation with 16, 8 and 4 nodes respectively
model.add(tf.keras.layers.Dense(16, name='Dense_layer_1'))
model.add(tf.keras.layers.Dense(8, name='Dense_layer_2'))
model.add(tf.keras.layers.Dense(4, name='Dense_layer_3'))

# Add another dense layer with the 2 nodes being the predicted future temperatures
model.add(tf.keras.layers.Dense(2, name='Output_layer'))

In [6]:
# Compile the model using RMSprop for optimisation and mean square error for loss
model.compile(tf.optimizers.RMSprop(0.001), loss='mse')
# Add a tensorboard callback for future reference
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [7]:
# Train the model over 50 epochs and add a 20% validation split to the input data for evaluation after training
model.fit(x=features.to_numpy(), y=target.to_numpy(), 
          epochs=50, callbacks=[tensorboard_callback], validation_split=0.2)

Epoch 1/50
190/190 [==============================] - 1s 3ms/step - loss: 320.1991 - val_loss: 23.4152
Epoch 2/50
190/190 [==============================] - 0s 2ms/step - loss: 12.2480 - val_loss: 6.4988
Epoch 3/50
190/190 [==============================] - 0s 2ms/step - loss: 5.0615 - val_loss: 5.0571
Epoch 4/50
190/190 [==============================] - 0s 2ms/step - loss: 3.6724 - val_loss: 3.8416
Epoch 5/50
190/190 [==============================] - 0s 2ms/step - loss: 3.0800 - val_loss: 3.4518
Epoch 6/50
190/190 [==============================] - 0s 2ms/step - loss: 2.7217 - val_loss: 3.2443
Epoch 7/50
190/190 [==============================] - 0s 2ms/step - loss: 2.4690 - val_loss: 2.8961
Epoch 8/50
190/190 [==============================] - 0s 2ms/step - loss: 2.2862 - val_loss: 2.5905
Epoch 9/50
190/190 [==============================] - 0s 2ms/step - loss: 2.1506 - val_loss: 2.4495
Epoch 10/50
190/190 [==============================] - 0s 2ms/step - loss: 2.0398 - val_loss: 2.

In [8]:
# Test the model on the 20% saved dataset for evaluation
loss = model.evaluate(features.to_numpy(), target.to_numpy())
# Output the loss on the unseen data
print('loss:', loss)

238/238 [==============================] - 0s 1ms/step - loss: 1.7509
loss: 1.7509491443634033
